In [1]:
#!pip install -U datasets optuna

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [2]:
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import set_seed, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score
import optuna
import pandas as pd

In [3]:
seed = 42
set_seed(seed)

In [4]:
train = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "rus", split="train")
# train_eng = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "ukr", split="train")

val = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "rus", split="dev")
# val_eng = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "eng", split="dev")
test = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "rus", split="test")

In [6]:
# train = concatenate_datasets([train, train_eng])

In [7]:
# val = concatenate_datasets([val, val_eng])

In [8]:
emotion_cols = ['anger', 'fear', 'joy', 'disgust', 'sadness', 'surprise']

In [9]:
def create_labels(examples):
    labels = []
    for i in range(len(examples['text'])):
        label = [float(examples[col][i]) for col in emotion_cols]
        labels.append(label)
    examples['labels'] = labels
    return examples

train = train.map(create_labels, batched=True)
val = val.map(create_labels, batched=True)
test = test.map(create_labels, batched=True)

Map:   0%|          | 0/2679 [00:00<?, ? examples/s]

In [10]:
model_name = "google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=False, max_length=512)

train_tokenized = train.map(tokenize_function, batched=True)
val_tokenized = val.map(tokenize_function, batched=True)

Map:   0%|          | 0/2679 [00:00<?, ? examples/s]

In [12]:
train_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = torch.sigmoid(torch.tensor(predictions)).numpy()
    y_pred = predictions > 0.5

    results = {}
    for average in ['micro', 'macro']:
        results[f'{average}_recall'] = recall_score(labels, y_pred, average=average, zero_division=0)
        results[f'{average}_precision'] = precision_score(labels, y_pred, average=average, zero_division=0)
        results[f'{average}_f1'] = f1_score(labels, y_pred, average=average, zero_division=0)

    return results

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 500)
    num_epochs = trial.suggest_int("num_train_epochs", 2, 7)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(emotion_cols),
        problem_type="multi_label_classification"
    )

    training_args = TrainingArguments(
        output_dir=f'./results/trial_{trial.number}',
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        logging_steps=50,
        eval_strategy="epoch",
        metric_for_best_model="eval_macro_f1",
        logging_dir=f'./logs/trial_{trial.number}',
        save_strategy="no",
        report_to=None,
        seed=seed,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
    )

    trainer.train()

    eval_results = trainer.evaluate()

    return eval_results["eval_macro_f1"]

In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("\nBest hyperparameters:")
print(study.best_params)
print(f"Best macro F1: {study.best_value:.4f}")

[I 2025-06-12 22:50:46,047] A new study created in memory with name: no-name-8dc3dd17-b8a6-4239-b0b6-998bec4f3f12
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.256900,0.224113,0.709845,0.761111,0.734584,0.720573,0.798664,0.749046
2,0.178300,0.192610,0.766839,0.800000,0.783069,0.781301,0.807663,0.787782
3,0.141200,0.194267,0.740933,0.877301,0.803371,0.752696,0.895880,0.809233
4,0.098000,0.175378,0.772021,0.866279,0.816438,0.788850,0.877467,0.827310
5,0.064400,0.183990,0.756477,0.884848,0.815642,0.770042,0.889119,0.822070
6,0.047900,0.194129,0.782383,0.877907,0.827397,0.798349,0.883040,0.834268
7,0.023900,0.194067,0.782383,0.862857,0.820652,0.798349,0.869818,0.828647


[I 2025-06-12 22:53:21,906] Trial 0 finished with value: 0.8286472173023985 and parameters: {'learning_rate': 4.0880000250683644e-05, 'batch_size': 8, 'weight_decay': 0.1896718711665509, 'warmup_steps': 94, 'num_train_epochs': 7}. Best is trial 0 with value: 0.8286472173023985.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.421900,0.384289,0.202073,0.866667,0.327731,0.172951,0.274962,0.207540
2,0.322400,0.292123,0.435233,0.923077,0.591549,0.427599,0.756774,0.533381


[I 2025-06-12 22:53:52,202] Trial 1 finished with value: 0.5333814333814334 and parameters: {'learning_rate': 1.097708908459796e-05, 'batch_size': 16, 'weight_decay': 0.02690975343175591, 'warmup_steps': 260, 'num_train_epochs': 2}. Best is trial 0 with value: 0.8286472173023985.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.346200,0.307444,0.295337,0.850746,0.438462,0.262101,0.568793,0.329893
2,0.228900,0.217714,0.658031,0.907143,0.762763,0.669283,0.907359,0.761017
3,0.163700,0.182761,0.735751,0.898734,0.809117,0.748827,0.900708,0.815086
4,0.131100,0.178386,0.740933,0.893750,0.810198,0.744508,0.890126,0.808349


[I 2025-06-12 23:20:29,766] Trial 18 finished with value: 0.8083493272696304 and parameters: {'learning_rate': 1.945216770154496e-05, 'batch_size': 16, 'weight_decay': 0.0032558539641831833, 'warmup_steps': 62, 'num_train_epochs': 4}. Best is trial 12 with value: 0.8324990546825356.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.638500,0.410631,0.056995,1.000000,0.107843,0.047009,0.166667,0.073333
2,0.372100,0.312186,0.383420,0.850575,0.528571,0.370661,0.719124,0.474571


[I 2025-06-12 23:20:55,365] Trial 19 finished with value: 0.474571137370383 and parameters: {'learning_rate': 2.8803463262445056e-05, 'batch_size': 32, 'weight_decay': 0.05853958492161683, 'warmup_steps': 286, 'num_train_epochs': 2}. Best is trial 12 with value: 0.8324990546825356.



Best hyperparameters:
{'learning_rate': 2.868318448990469e-05, 'batch_size': 8, 'weight_decay': 0.004901939692684662, 'warmup_steps': 87, 'num_train_epochs': 4}
Best macro F1: 0.8325


In [18]:
Best hyperparameters:
{'learning_rate': 1.952733015072204e-05, 'batch_size': 16, 'weight_decay': 0.28658541378102453, 'warmup_steps': 471, 'num_train_epochs': 7}
Best macro F1: 0.9054

SyntaxError: invalid syntax (4240920430.py, line 1)

In [19]:
best_params = study.best_params
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(emotion_cols),
    problem_type="multi_label_classification"
)

training_args = TrainingArguments(
    output_dir='./best_model',
    num_train_epochs=best_params["num_train_epochs"],
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    learning_rate=best_params["learning_rate"],
    weight_decay=best_params["weight_decay"],
    warmup_steps=best_params["warmup_steps"],
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="no",
    logging_dir='./logs/best_model',
    seed=seed,
)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Micro Recall,Micro Precision,Micro F1,Macro Recall,Macro Precision,Macro F1
1,0.262500,0.234877,0.637306,0.814570,0.715116,0.636522,0.823705,0.704607
2,0.166100,0.189025,0.761658,0.825843,0.792453,0.775043,0.833577,0.798757
3,0.130500,0.164405,0.761658,0.913043,0.830508,0.765201,0.920944,0.831661
4,0.090700,0.160479,0.777202,0.892857,0.831025,0.780635,0.899369,0.833552


TrainOutput(global_step=1340, training_loss=0.1875365776802177, metrics={'train_runtime': 87.6756, 'train_samples_per_second': 122.223, 'train_steps_per_second': 15.284, 'total_flos': 248538900917736.0, 'train_loss': 0.1875365776802177, 'epoch': 4.0})

In [20]:
def find_best_threshold(model, val_dataset, thresholds=np.arange(0.1, 0.9, 0.05)):
    model.eval()

    predictions = trainer.predict(val_dataset)
    probs = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
    true_labels = predictions.label_ids

    optimal_thresholds = {}
    best_f1_scores = {}
    
    print("Finding optimal thresholds for each emotion:")
    print("=" * 60)
    
    # Find optimal threshold for each emotion separately
    for i, emotion in enumerate(emotion_cols):
        best_threshold = 0.5
        best_f1 = 0
        
        print(f"\n{emotion.upper()}:")
        print("-" * 30)
        
        for threshold in thresholds:
            # Binary predictions for this emotion only
            y_pred_emotion = (probs[:, i] > threshold).astype(int)
            y_true_emotion = true_labels[:, i]
            
            # Calculate F1 for this emotion
            f1_emotion = f1_score(y_true_emotion, y_pred_emotion, zero_division=0)
            
            print(f"Threshold {threshold:.2f}: F1 = {f1_emotion:.4f}")
            
            if f1_emotion > best_f1:
                best_f1 = f1_emotion
                best_threshold = threshold
        
        optimal_thresholds[emotion] = best_threshold
        best_f1_scores[emotion] = best_f1
        
        print(f"Best threshold for {emotion}: {best_threshold:.2f} (F1: {best_f1:.4f})")
    return optimal_thresholds

In [21]:
def find_best_threshold(model, val_dataset, thresholds=np.arange(0.1, 0.9, 0.05)):
    model.eval()
    predictions = trainer.predict(val_dataset)
    probs = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
    true_labels = predictions.label_ids
    best_threshold = 0.5
    best_f1 = 0
    print("Threshold optimization:")
    for threshold in thresholds:
        y_pred = probs > threshold
        f1_macro = f1_score(true_labels, y_pred, average='macro', zero_division=0)
        print(f"Threshold {threshold:.2f}: Macro F1 = {f1_macro:.4f}")
        if f1_macro > best_f1:
            best_f1 = f1_macro
            best_threshold = threshold
    print(f"\nBest threshold: {best_threshold:.2f} (Macro F1: {best_f1:.4f})")
    return best_threshold

In [22]:
best_threshold = find_best_threshold(model, val_tokenized)

Threshold optimization:
Threshold 0.10: Macro F1 = 0.7770
Threshold 0.15: Macro F1 = 0.8143
Threshold 0.20: Macro F1 = 0.8246
Threshold 0.25: Macro F1 = 0.8375
Threshold 0.30: Macro F1 = 0.8383
Threshold 0.35: Macro F1 = 0.8480
Threshold 0.40: Macro F1 = 0.8481
Threshold 0.45: Macro F1 = 0.8437
Threshold 0.50: Macro F1 = 0.8336
Threshold 0.55: Macro F1 = 0.8290
Threshold 0.60: Macro F1 = 0.8329
Threshold 0.65: Macro F1 = 0.8325
Threshold 0.70: Macro F1 = 0.8344
Threshold 0.75: Macro F1 = 0.8222
Threshold 0.80: Macro F1 = 0.8157
Threshold 0.85: Macro F1 = 0.7954

Best threshold: 0.40 (Macro F1: 0.8481)


In [23]:
# test_tokenized = test.map(tokenize_function, batched=True)
# test_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# test_predictions = trainer.predict(test_tokenized)
# test_probs = torch.sigmoid(torch.tensor(test_predictions.predictions)).numpy()
# test_pred_labels = test_probs > best_threshold
# true_test_labels = test_predictions.label_ids

In [24]:
test_tokenized = test.map(tokenize_function, batched=True)
test_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

test_predictions = trainer.predict(test_tokenized)
test_probs = torch.sigmoid(torch.tensor(test_predictions.predictions)).numpy()
test_pred_labels = np.zeros_like(test_probs)
for i, emotion in enumerate(emotion_cols):
    test_pred_labels[:, i] = (test_probs[:, i] > 0.5).astype(int)
true_test_labels = test_predictions.label_ids

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [25]:
# print(f"\nTest Results with optimal threshold ({best_threshold:.2f}):")
for average in ['micro', 'macro']:
    recall = recall_score(true_test_labels, test_pred_labels, average=average, zero_division=0)
    precision = precision_score(true_test_labels, test_pred_labels, average=average, zero_division=0)
    f1 = f1_score(true_test_labels, test_pred_labels, average=average, zero_division=0)
    print(f'{average.upper()} recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}')

print(f"\nPer-class Results:")
class_recall = recall_score(true_test_labels, test_pred_labels, average=None, zero_division=0)
class_precision = precision_score(true_test_labels, test_pred_labels, average=None, zero_division=0)
class_f1 = f1_score(true_test_labels, test_pred_labels, average=None, zero_division=0)

for i, emotion in enumerate(emotion_cols):
    print(f'{emotion.upper()}: recall: {round(class_recall[i], 4)}, precision: {round(class_precision[i], 4)}, f1: {round(class_f1[i], 4)}')

print(f"\nClass distribution in test set:")
for i, emotion in enumerate(emotion_cols):
    true_count = int(true_test_labels[:, i].sum())
    pred_count = int(test_pred_labels[:, i].sum())
    total = len(true_test_labels)
    print(f'{emotion.upper()}: true: {true_count}/{total} ({true_count/total:.1%}), predicted: {pred_count}/{total} ({pred_count/total:.1%})')

MICRO recall: 0.7777, precision: 0.8712, f1: 0.8218
MACRO recall: 0.7782, precision: 0.8762, f1: 0.8225

Per-class Results:
ANGER: recall: 0.7611, precision: 0.839, f1: 0.7981
FEAR: recall: 0.8611, precision: 0.8774, f1: 0.8692
JOY: recall: 0.8238, precision: 0.8933, f1: 0.8571
DISGUST: recall: 0.7049, precision: 0.9451, f1: 0.8075
SADNESS: recall: 0.7376, precision: 0.8889, f1: 0.8062
SURPRISE: recall: 0.7805, precision: 0.8136, f1: 0.7967

Class distribution in test set:
ANGER: true: 452/2000 (22.6%), predicted: 410/2000 (20.5%)
FEAR: true: 216/2000 (10.8%), predicted: 212/2000 (10.6%)
JOY: true: 386/2000 (19.3%), predicted: 356/2000 (17.8%)
DISGUST: true: 244/2000 (12.2%), predicted: 182/2000 (9.1%)
SADNESS: true: 282/2000 (14.1%), predicted: 234/2000 (11.7%)
SURPRISE: true: 246/2000 (12.3%), predicted: 236/2000 (11.8%)


In [36]:
np.save('bert.npy', test_probs)



In [37]:
arr_loaded

array([[0.01563433, 0.9697105 , 0.01789523, 0.0131724 , 0.01356052,
        0.01505389],
       [0.00515464, 0.00593253, 0.00646108, 0.00496511, 0.00630429,
        0.00407498],
       [0.98380107, 0.01522074, 0.01516229, 0.02613696, 0.02344373,
        0.01456004],
       ...,
       [0.09306297, 0.00388846, 0.00349513, 0.00452616, 0.03717936,
        0.00292134],
       [0.0076504 , 0.02186554, 0.02283526, 0.01351371, 0.01696276,
        0.96584934],
       [0.00909997, 0.01079418, 0.98410505, 0.00929993, 0.0122868 ,
        0.00937763]], dtype=float32)